#simple Aggregations df

In [2]:
import os

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [5]:
!pip install pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 21.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767582 sha256=329b35e098021b1b973b357317daf719bf75c7d4f601af5515a54f04c42da55b
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [6]:
import pyspark

In [7]:
import findspark

In [8]:
findspark .init()

In [9]:
from pyspark.sql import SparkSession

In [10]:
if __name__=="__main__":
  spark = SparkSession.builder \
      .appName("myapplication") \
      .master("local[*]") \
      .getOrCreate()

In [11]:
spark

In [12]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DecimalType,DateType,TimestampType

In [13]:
cust_schema=StructType([StructField("cid",IntegerType()),
                        StructField("fname",StringType()),
                        StructField("lname",StringType()),
                        StructField("age",IntegerType()),
                        StructField("desig",StringType())])

In [14]:
cust_df=spark.read\
            .option("header","true")\
            .schema(cust_schema)\
            .csv("/content/custs_with_header.csv")

In [15]:
cust_df.printSchema()

root
 |-- cid: integer (nullable = true)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- desig: string (nullable = true)



In [16]:
from pyspark.sql.functions import sum,avg,min,max,count,col,countDistinct,round

In [17]:
cust_df.select(sum("age"),round(avg("age"),2),min("age"),max("age"),count("age"),countDistinct(col("desig"))).show()

+--------+------------------+--------+--------+----------+---------------------+
|sum(age)|round(avg(age), 2)|min(age)|max(age)|count(age)|count(DISTINCT desig)|
+--------+------------------+--------+--------+----------+---------------------+
|  485632|             48.57|      21|      75|      9999|                   50|
+--------+------------------+--------+--------+----------+---------------------+



In [18]:
from pyspark.sql import functions as f

In [19]:
cust_df.select(f.sum("age"),f.round(f.avg("age"),2),f.min("age"),f.max("age"),f.count("age"),f.countDistinct("age")).show()

+--------+------------------+--------+--------+----------+-------------------+
|sum(age)|round(avg(age), 2)|min(age)|max(age)|count(age)|count(DISTINCT age)|
+--------+------------------+--------+--------+----------+-------------------+
|  485632|             48.57|      21|      75|      9999|                 55|
+--------+------------------+--------+--------+----------+-------------------+



Multiple aggregations

In [20]:
tnx_schema=StructType([StructField("tnx_id",IntegerType()),
                       StructField("tnx_dt",DateType()),
                       StructField("cid",IntegerType()),
                       StructField("amount",DecimalType(10,2)),
                       StructField("prod_cat",StringType()),
                       StructField("prod",StringType()),
                       StructField("city",StringType()),
                       StructField("state",StringType()),
                       StructField("mode",StringType())])

In [21]:
tnx_df=spark.read\
           .option("header",True)\
           .schema(tnx_schema)\
           .option("dateFormat","MM-dd-yyyy")\
           .csv("/content/txns_with_header.csv")

In [22]:
tnx_df.show(5)

+------+----------+-------+------+------------------+--------------------+-----------+----------+------+
|tnx_id|    tnx_dt|    cid|amount|          prod_cat|                prod|       city|     state|  mode|
+------+----------+-------+------+------------------+--------------------+-----------+----------+------+
|     0|2011-06-26|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee|credit|
|     1|2011-05-26|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California|credit|
|     2|2011-06-01|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California|credit|
|     3|2011-06-05|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin|credit|
|     4|2011-12-17|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee|credit|
+------+----------+-------+------+------------------+--------------------+-----------+----------+------+
only showing top 5 rows



In [23]:
tnx_df.groupby('state').sum("amount").orderBy(sum("amount"),ascending=False).show(5)

+----------+-----------+
|     state|sum(amount)|
+----------+-----------+
|California|  702346.23|
|     Texas|  541267.24|
|   Florida|  274858.95|
|      Ohio|  188613.87|
|    Oregon|  187052.05|
+----------+-----------+
only showing top 5 rows



In [24]:
total_sale=f.round(f.sum("amount"),0).alias("total_sales")
tnx_df\
.groupby("state")\
.agg(
    total_sale,\
     round(f.avg("amount"),2).alias("average_sales"),\
     f.min("amount").alias("minimum_sales"),\
     f.max("amount").alias("maximum_sales"),\
     ).show()

+--------------------+-----------+-------------+-------------+-------------+
|               state|total_sales|average_sales|minimum_sales|maximum_sales|
+--------------------+-----------+-------------+-------------+-------------+
|                Utah|      92071|       104.51|         5.18|       199.85|
|              Hawaii|      42950|       101.54|         6.75|       199.82|
|           Minnesota|      92498|       103.47|         5.08|       199.98|
|                Ohio|     188614|       101.35|         5.22|       199.94|
|              Oregon|     187052|       102.44|         5.09|       199.83|
|               Texas|     541267|       101.38|         5.00|       199.91|
|        Pennsylvania|      90443|       102.78|         5.01|       198.97|
|         Connecticut|      91084|        99.98|         5.32|       199.93|
|            Nebraska|      91556|       101.96|         5.15|       199.63|
|              Nevada|      95418|       105.43|         5.08|       199.67|

In [32]:
total_sale=f.round(f.sum("amount"),0).alias("total_sales")
tnx_df\
.groupby("state","city")\
.agg(
    total_sale,\
     round(f.avg("amount"),2).alias("average_sales"),\
     f.min("amount").alias("minimum_sales"),\
     f.max("amount").alias("maximum_sales"),\
     ).orderBy("state","city",ascending=[True,True])\
     .show()

+----------+-------------+-----------+-------------+-------------+-------------+
|     state|         city|total_sales|average_sales|minimum_sales|maximum_sales|
+----------+-------------+-----------+-------------+-------------+-------------+
|   Alabama|   Birmingham|      20612|        96.32|         5.57|       199.39|
|   Alabama|   Huntsville|      46623|       105.72|         5.92|       199.21|
|   Alabama|   Montgomery|      48780|       102.26|         5.20|       199.94|
|   Arizona|      Gilbert|      40408|       101.02|         5.71|       200.00|
|   Arizona|      Phoenix|      45920|       102.27|         5.31|       199.58|
|   Arizona|   Scottsdale|      44963|       103.13|         5.01|       199.99|
|California|      Anaheim|      45263|       102.17|         5.06|       199.40|
|California|     Berkeley|      43734|       101.00|         6.05|       199.88|
|California|      Fremont|      47499|       103.71|         5.19|       199.88|
|California|       Irvine|  

In [89]:
total_sale=f.round(f.sum("amount"),0).alias("total_sales")
tnx_df\
.groupby("state","city")\
.agg(
    total_sale,\
     round(f.avg("amount"),2).alias("average_sales"),\
     f.min("amount").alias("minimum_sales"),\
     f.max("amount").alias("maximum_sales"),\
     ).orderBy("state","total_sales",ascending=[True,False])\
     .show(100)

+--------------------+----------------+-----------+-------------+-------------+-------------+
|               state|            city|total_sales|average_sales|minimum_sales|maximum_sales|
+--------------------+----------------+-----------+-------------+-------------+-------------+
|             Alabama|      Montgomery|      48780|       102.26|         5.20|       199.94|
|             Alabama|      Huntsville|      46623|       105.72|         5.92|       199.21|
|             Alabama|      Birmingham|      20612|        96.32|         5.57|       199.39|
|             Arizona|         Phoenix|      45920|       102.27|         5.31|       199.58|
|             Arizona|      Scottsdale|      44963|       103.13|         5.01|       199.99|
|             Arizona|         Gilbert|      40408|       101.02|         5.71|       200.00|
|          California|      Long Beach|      52687|       109.31|         5.24|       199.64|
|          California|         Oakland|      49765|       10

In [91]:
total_sale=f.round(f.sum("amount"),0).alias("total_sales")
tnx_df\
.groupby("state","city")\
.agg(
    total_sale,\
     round(f.avg("amount"),2).alias("average_sales"),\
     f.min("amount").alias("minimum_sales"),\
     f.max("amount").alias("maximum_sales"),\
     ).orderBy("state","city",ascending=[True,True])\
     .where(col("state").isin('California','Arizona','Texas','Kentucky'))\
     .show()

+----------+-------------+-----------+-------------+-------------+-------------+
|     state|         city|total_sales|average_sales|minimum_sales|maximum_sales|
+----------+-------------+-----------+-------------+-------------+-------------+
|   Arizona|      Gilbert|      40408|       101.02|         5.71|       200.00|
|   Arizona|      Phoenix|      45920|       102.27|         5.31|       199.58|
|   Arizona|   Scottsdale|      44963|       103.13|         5.01|       199.99|
|California|      Anaheim|      45263|       102.17|         5.06|       199.40|
|California|     Berkeley|      43734|       101.00|         6.05|       199.88|
|California|      Fremont|      47499|       103.71|         5.19|       199.88|
|California|       Irvine|      48039|       106.52|         5.51|       199.80|
|California|   Long Beach|      52687|       109.31|         5.24|       199.64|
|California|  Los Angeles|      45167|        98.62|         5.03|       199.67|
|California|      Oakland|  

##window functions usage in data frame

In [51]:
from pyspark.sql import Window
#For Window W is capital

In [48]:
total_sale_statewise_df=tnx_df.groupby("state","city")\
                             .agg(round(sum("amount"),0).alias("total_sales"))\
                             .orderBy("state","city",ascending=[True,True])

In [49]:
total_sale_statewise_df.show()

+----------+-------------+-----------+
|     state|         city|total_sales|
+----------+-------------+-----------+
|   Alabama|   Birmingham|      20612|
|   Alabama|   Huntsville|      46623|
|   Alabama|   Montgomery|      48780|
|   Arizona|      Gilbert|      40408|
|   Arizona|      Phoenix|      45920|
|   Arizona|   Scottsdale|      44963|
|California|      Anaheim|      45263|
|California|     Berkeley|      43734|
|California|      Fremont|      47499|
|California|       Irvine|      48039|
|California|   Long Beach|      52687|
|California|  Los Angeles|      45167|
|California|      Oakland|      49765|
|California|       Orange|      47215|
|California|     Pasadena|      47061|
|California|   Sacramento|      47731|
|California|    San Diego|      43127|
|California|San Francisco|      45341|
|California|     San Jose|      47603|
|California|    Santa Ana|      48209|
+----------+-------------+-----------+
only showing top 20 rows



In [53]:
window_for_running_total=Window.partitionBy("state")\
                               .orderBy("city")\
                               .rowsBetween(Window.unboundedPreceding,Window.currentRow)
#For Window W is capital and in rows between we dont use quotations

In [55]:
calculating_runningtotal_df=total_sale_statewise_df\
                            .withColumn("Running_total",sum("total_sales").over(window_for_running_total))\
                            .orderBy("state","city",ascending=[True,True])
#in over clause we donot use quotation

In [56]:
calculating_runningtotal_df.show(20)

+----------+-------------+-----------+-------------+
|     state|         city|total_sales|Running_total|
+----------+-------------+-----------+-------------+
|   Alabama|   Birmingham|      20612|        20612|
|   Alabama|   Huntsville|      46623|        67235|
|   Alabama|   Montgomery|      48780|       116015|
|   Arizona|      Gilbert|      40408|        40408|
|   Arizona|      Phoenix|      45920|        86328|
|   Arizona|   Scottsdale|      44963|       131291|
|California|      Anaheim|      45263|        45263|
|California|     Berkeley|      43734|        88997|
|California|      Fremont|      47499|       136496|
|California|       Irvine|      48039|       184535|
|California|   Long Beach|      52687|       237222|
|California|  Los Angeles|      45167|       282389|
|California|      Oakland|      49765|       332154|
|California|       Orange|      47215|       379369|
|California|     Pasadena|      47061|       426430|
|California|   Sacramento|      47731|       4

In [61]:
calculating_runningtotal_df_avg=total_sale_statewise_df\
                            .withColumn("avg_Running_total",avg("total_sales").over(window_for_running_total))\
                            .orderBy("state","city",ascending=[True,True])
#in over clause we donot use quotation

In [62]:
calculating_runningtotal_df_avg.show()

+----------+-------------+-----------+-----------------+
|     state|         city|total_sales|avg_Running_total|
+----------+-------------+-----------+-----------------+
|   Alabama|   Birmingham|      20612|       20612.0000|
|   Alabama|   Huntsville|      46623|       33617.5000|
|   Alabama|   Montgomery|      48780|       38671.6667|
|   Arizona|      Gilbert|      40408|       40408.0000|
|   Arizona|      Phoenix|      45920|       43164.0000|
|   Arizona|   Scottsdale|      44963|       43763.6667|
|California|      Anaheim|      45263|       45263.0000|
|California|     Berkeley|      43734|       44498.5000|
|California|      Fremont|      47499|       45498.6667|
|California|       Irvine|      48039|       46133.7500|
|California|   Long Beach|      52687|       47444.4000|
|California|  Los Angeles|      45167|       47064.8333|
|California|      Oakland|      49765|       47450.5714|
|California|       Orange|      47215|       47421.1250|
|California|     Pasadena|     

In [63]:
tnx_df.show(5)

+------+----------+-------+------+------------------+--------------------+-----------+----------+------+
|tnx_id|    tnx_dt|    cid|amount|          prod_cat|                prod|       city|     state|  mode|
+------+----------+-------+------+------------------+--------------------+-----------+----------+------+
|     0|2011-06-26|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee|credit|
|     1|2011-05-26|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California|credit|
|     2|2011-06-01|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California|credit|
|     3|2011-06-05|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin|credit|
|     4|2011-12-17|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee|credit|
+------+----------+-------+------+------------------+--------------------+-----------+----------+------+
only showing top 5 rows



In [82]:
totalsales_for_product_df=tnx_df.groupBy("prod_cat","prod")\
                                .agg(round(sum("amount"),0).alias("productsales"))\
                                .orderBy("prod_cat","prod",ascending=[True,True])

In [84]:
running_total_for_product=Window.partitionBy("prod_cat")\
                                .orderBy("prod")\
                                .rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [85]:
runningtotal_for_productcategory_df=totalsales_for_product_df.withColumn("productcat_running_total",sum("productsales").over(running_total_for_product))\
                                                                         .orderBy("prod_cat","prod",ascending=[True,True])

In [86]:
runningtotal_for_productcategory_df.show()

+------------------+--------------------+------------+------------------------+
|          prod_cat|                prod|productsales|productcat_running_total|
+------------------+--------------------+------------+------------------------+
|        Air Sports|           Air Suits|       19372|                   19372|
|        Air Sports|        Hang Gliding|       38759|                   58131|
|        Air Sports|          Parachutes|       41186|                   99317|
|     Combat Sports|              Boxing|       44517|                   44517|
|     Combat Sports|             Fencing|       40604|                   85121|
|     Combat Sports|        Martial Arts|       39277|                  124398|
|     Combat Sports|           Wrestling|       40333|                  164731|
|           Dancing|         Ballet Bars|       42604|                   42604|
|Exercise & Fitness| Abdominal Equipment|       43304|                   43304|
|Exercise & Fitness|Cardio Machine Ac...

In [124]:
rankWindow_for_product=Window.partitionBy("prod_cat")\
                            .orderBy("productsales")

In [127]:
top_3_ranks_for_productcategory=totalsales_for_product_df.withColumn("Rank",f.rank()\
                                                                     .over(rankWindow_for_product))\
                                                                     .orderBy("prod_cat")
#order should be mentioned in this function by default descending is true

In [128]:
top_3_ranks_for_productcategory.where("Rank < 4").show()

+------------------+--------------------+------------+----+
|          prod_cat|                prod|productsales|Rank|
+------------------+--------------------+------------+----+
|        Air Sports|           Air Suits|       19372|   1|
|        Air Sports|        Hang Gliding|       38759|   2|
|        Air Sports|          Parachutes|       41186|   3|
|     Combat Sports|        Martial Arts|       39277|   1|
|     Combat Sports|           Wrestling|       40333|   2|
|     Combat Sports|             Fencing|       40604|   3|
|           Dancing|         Ballet Bars|       42604|   1|
|Exercise & Fitness|      Exercise Bands|       37680|   1|
|Exercise & Fitness|Weightlifting Gloves|       38439|   2|
|Exercise & Fitness|            Gym Mats|       39275|   3|
|             Games|          Dice Games|       40397|   2|
|             Games|          Card Games|       40839|   3|
|             Games|            Dominoes|       39585|   1|
|        Gymnastics|    Gymnastics Rings